In [9]:
# !pip install matplotlib torchvision



In [10]:
model_name = "ResNetSE_label_smoothing_ReduceLR"

In [11]:
print("hello")

hello


In [12]:
import matplotlib.pyplot as plt

import numpy as np

from collections import namedtuple


import pickle

from tqdm import tqdm

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.nn.init as init
import csv

In [13]:
import torchvision
import torchvision.transforms as transforms
from torch.optim import SGD, lr_scheduler, Adam
from torch.utils.data import TensorDataset

In [14]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [15]:
device

device(type='cuda')

In [17]:


# SE Block
class SEBlock(nn.Module):
    def __init__(self, channels, reduction=16):
        super().__init__()
        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        self.fc = nn.Sequential(
            nn.Linear(channels, channels // reduction),
            nn.ReLU(inplace=True),
            nn.Linear(channels // reduction, channels),
            nn.Sigmoid()
        )

    def forward(self, x):
        b, c, _, _ = x.size()
        y = self.avg_pool(x).view(b, c)
        y = self.fc(y).view(b, c, 1, 1)
        return x * y

# Modified BasicBlock with SE
class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, in_channels, out_channels, stride=1):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=True)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=True)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.se = SEBlock(out_channels)  # Adding SE Block
        
        self.shortcut = nn.Sequential()
        if stride != 1 or in_channels != self.expansion * out_channels:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, self.expansion * out_channels, kernel_size=1, stride=stride, bias=True),
                nn.BatchNorm2d(self.expansion * out_channels)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out = self.se(out)  # Apply SE
        out += self.shortcut(x)
        out = F.relu(out)
        return out

# Modified ResNet with SE and adjusted depth
class ResNet(nn.Module):
    def __init__(self, block, num_blocks, num_classes=10):
        super().__init__()
        self.in_channels = 64

        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=True)
        self.bn1 = nn.BatchNorm2d(64)
        self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 254, num_blocks[2], stride=2)
        self.linear = nn.Linear(254 * block.expansion, num_classes)

    def _make_layer(self, block, out_channels, num_blocks, stride):
        strides = [stride] + [1] * (num_blocks - 1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_channels, out_channels, stride))
            self.in_channels = out_channels * block.expansion
        return nn.Sequential(*layers)

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = F.avg_pool2d(out, 8)
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out

# Instantiate the model with adjusted num_blocks [4,5,3]
def ResNetSE():
    return ResNet(BasicBlock, [4,5,3])


# Training Setup

model = ResNetSE().to(device)
print(f"Parameters: {sum(p.numel() for p in model.parameters())/1e6:.2f}M")

# Data Augmentation
transform_train = transforms.Compose([
	transforms.RandomCrop(32, padding=4),
	transforms.RandomHorizontalFlip(),
	transforms.ToTensor(),
	transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

transform_test = transforms.Compose([
	transforms.ToTensor(),
	transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])


test_batch_size = 128
train_batch_size = 128


# Download and load the training dataset
trainset = torchvision.datasets.CIFAR10(
	root='./data',		  # Path to store the dataset
	train=True,			 # Load the training set
	download=True,		  # Download the dataset if it doesn't exist
	transform=transform_train	 # Apply the defined transformation
)

# Create a DataLoader for the training set
trainloader = torch.utils.data.DataLoader(
	trainset,			   # Dataset to load
	batch_size=train_batch_size,		  # Batch size
	shuffle=True,		   # Shuffle the data
	num_workers=4		   # Number of subprocesses to use for data loading
)

# Download and load the test dataset
testset = torchvision.datasets.CIFAR10(
	root='./data',		  # Path to store the dataset
	train=False,			# Load the test set
	download=True,		  # Download the dataset if it doesn't exist
	transform=transform_test	 # Apply the defined transformation
)

# Create a DataLoader for the test set
testloader = torch.utils.data.DataLoader(
	testset,				# Dataset to load
	batch_size=train_batch_size,		  # Batch size
	shuffle=False,		  # Do not shuffle the data
	num_workers=4		   # Number of subprocesses to use for data loading
)



# def unpickle(file):

# 	with open(file, 'rb') as fo:
# 		dict = pickle.load(fo, encoding='bytes')
# 	return dict


# def convert_dict_to_namedtuple(d):
#   out = []
#   batch_labbel = d[b'batch_label']
#   for idx, item in enumerate(d[b'data']):
#	 tupl = Cifar10(d[b'data'][idx], d[b'labels'][idx])
#	 out.append(tupl)

#   return out






# from google.colab import drive
# drive.mount('/content/drive')

# # Select three random numbers between 1 & 100
# random_numbers = np.random.randint(1, 101, size=3)
# samples_images = []

# for i in random_numbers:
#   samples_images.append(trainset[i])


# # Create a figure and subplots
# fig, axes = plt.subplots(1, len(samples_images), figsize=(10, 1))  # 1 row, 3 columns

# # Display the images

# for i in range(len(samples_images)):
#   # The transpose(1, 2, 0) operation rearranges the dimensions so that the channels (color information) come last, which is typically the expected format for image display.

#   # reshape(3, 32, 32) assumes the image is a color image (3 channels - Red, Green, Blue) with a width and height of 32 pixels each.

#   axes[i].imshow(samples_images[i][0].cpu().view(3, 32, 32).permute(1, 2, 0))

#   # Label the subplots
#   axes[i].set_title("Label: " + str(samples_images[i][1].item()))

#   # Remove the axis ticks
#   axes[i].axis('off')


# # Adjust spacing between subplots
# plt.tight_layout()

# # Show the plot
# plt.show()





# Training Config
optimizer = SGD(model.parameters(), lr=0.0001, weight_decay=5e-3)
scheduler = lr_scheduler.ReduceLROnPlateau(
    optimizer,
    mode='min',          # because you're tracking validation loss
    factor=0.1,           # reduce LR by 10x
    patience=5,           # wait 5 epochs without improvement
    threshold=1e-4,       # minimum change in val_loss to qualify as improvement
    cooldown=2,           # stop LR reduction for 2 epochs after each reduction
    min_lr=1e-5,          # don’t reduce below this
    verbose=True          # print when LR is reduced
)
cross_entropy_loss = nn.CrossEntropyLoss(label_smoothing=0.1)





Parameters: 4.98M


/home/nd2745/.local/lib/python3.9/site-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


In [18]:
print(f"Parameters: {sum(p.numel() for p in model.parameters())/1e6:.2f}M")

Parameters: 4.98M


In [19]:
# Initialize the model's weights with Xavier initialization
def init_weights_xavier(m):
	if isinstance(m, nn.Linear) or isinstance(m, nn.Conv2d):  # Check if the layer is Linear or Conv2d
		init.xavier_uniform_(m.weight)  # Apply Xavier uniform initialization to weights
		if m.bias is not None:
			nn.init.zeros_(m.bias) # Initialize biases to zero (optional)

# Apply the initialization function to the model
model.apply(init_weights_xavier)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (se): SEBlock(
        (avg_pool): AdaptiveAvgPool2d(output_size=1)
        (fc): Sequential(
          (0): Linear(in_features=64, out_features=4, bias=True)
          (1): ReLU(inplace=True)
          (2): Linear(in_features=4, out_features=64, bias=True)
          (3): Sigmoid()
        )
      )
      (shortcut): Sequential()
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padd

In [20]:
from heapq import heapify, heappush, heappop
top_5_accuracy = [0]

In [23]:
training_loss_history = []
test_loss_history = []
training_accuracy_history = []
validation_accuracy_history = []

print(f"Trainable Parameters: {sum(p.numel() for p in model.parameters() if p.requires_grad)/1e6:.2f}M")
print(f"Training started >>> ")
print(top_5_accuracy)

for epoch in tqdm(range(200)):
    model.train()
    total_loss = 0

    for input, labels in trainloader:
        input, labels = input.to(device).float().view(-1, 3, 32, 32), labels.to(device)
        optimizer.zero_grad()
        outputs = model(input)
        loss = cross_entropy_loss(outputs, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    avg_train_loss = total_loss / len(trainloader)
    training_loss_history.append(avg_train_loss)

    # Validation
    model.eval()
    correct = 0
    total = 0
    total_test_loss = 0

    with torch.no_grad():
        for inputs, labels in testloader:
            inputs, labels = inputs.to(device).view(-1, 3, 32, 32), labels.to(device)
            outputs = model(inputs)
            test_loss = cross_entropy_loss(outputs, labels)
            total_test_loss += test_loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    avg_val_loss = total_test_loss / len(testloader)

    test_loss_history.append(avg_val_loss)
    validation_accuracy_history.append(accuracy)
    training_accuracy_history.append(100 * correct / total)

    scheduler.step(avg_val_loss)

    # Save top 5 checkpoints
    if accuracy > top_5_accuracy[0]:
        if len(top_5_accuracy) >= 5:
            heappop(top_5_accuracy)
        torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': loss,
        }, f'./checkpoints_{model_name}/resnet_full_checkpoint_{accuracy:.2f}.pth')
        heappush(top_5_accuracy, accuracy)

    print(f"Epoch {epoch+1}, Train Loss: {avg_train_loss:.3f}, Val Loss: {avg_val_loss:.3f}, Test Accuracy: {accuracy:.2f}%")


Trainable Parameters: 4.98M
Training started >>> 
[0]


  0%|          | 1/200 [00:26<1:27:53, 26.50s/it]

Epoch 1, Train Loss: 2.221, Val Loss: 2.173, Test Accuracy: 20.81%


  1%|          | 2/200 [00:52<1:26:51, 26.32s/it]

Epoch 2, Train Loss: 2.174, Val Loss: 2.135, Test Accuracy: 21.52%


...

Epoch 198, Train Loss: 1.447, Val Loss: 1.440, Test Accuracy: 57.34%


100%|█████████▉| 199/200 [1:17:22<00:13, 13.77s/it]

Epoch 199, Train Loss: 1.447, Val Loss: 1.439, Test Accuracy: 57.32%


100%|██████████| 200/200 [1:17:36<00:00, 23.28s/it]

Epoch 200, Train Loss: 1.445, Val Loss: 1.438, Test Accuracy: 57.42%


In [24]:
def unpickle(file):

	with open(file, 'rb') as fo:
		dict = pickle.load(fo, encoding='bytes')
	return dict

nolabel_file = './cifar_test_nolabel.pkl'

no_label = unpickle(nolabel_file)

transform_unseen = transforms.Compose([
    transforms.ToTensor(),                 # Scale to [0, 1]
    transforms.Normalize(mean=[0.4914, 0.4822, 0.4465], std=[0.2470, 0.2435, 0.2616])  # Normalize
])

class Cifar10(torch.utils.data.Dataset):

    def __init__(self, data):
        super().__init__()
        label = data.get(b'labels', None)
        self.ids = [int(x) for x in data[b'ids']]
        data = [transform_unseen(x) for x in data[b'data']]
        # self.ids_to_data = {}
        self.x_data = torch.stack(data).to(device)

        
        # self._construct_id_hash()

        if label is not None:
            self.y_data = torch.tensor(label, dtype=torch.uint8).to(device)
        else:
            self.y_data = None
    
    def __len__(self):
        return len(self.x_data)  # required
    
    def __getitem__(self, idx):
        data = self.x_data[idx]
        return data

def visualize(input, counter):
    # Convert the tensor to (H, W, C) format using .permute()
    if counter < 5:
        print(input.shape)
        image_np = input[0].permute(1, 2, 0).cpu().numpy()
        
        # Plot the image
        plt.imshow(image_np)
        plt.axis('off')  # Optional: turn off axes for better visualization
        plt.show()

In [ ]:
# load_model_name = "resnet_full_checkpoint_93.65.pth"
model = ResNetSE().to(device)  # Replace with your model class and arguments
model_path = "/scratch/nd2745/checkpoints_ResNetSE_label_smoothing"
checkpoint = torch.load(f"{model_path}/{load_model_name}")
model.load_state_dict(checkpoint['model_state_dict'])
model.eval()

# Prepare unlabeled data and DataLoader
unlabeled_data = no_label  # Replace with your input data (e.g., images or tensors)
dataset = Cifar10(unlabeled_data)
unlabelled_dataloader = torch.utils.data.DataLoader(dataset, batch_size=32, shuffle=False)


# Run inference on unlabeled data
counter = 0
predictions = []
with torch.no_grad():
    for batch in unlabelled_dataloader:
        # for item in batch:
        #     ids.append(dataset.get_id(item))
        
        batch = batch.to(device).float().view(-1, 3, 32, 32)
        counter += 1
        visualize(batch, counter)
        outputs = model(batch)
        predicted_labels = torch.argmax(outputs, dim=1)
        predictions.extend(predicted_labels.cpu().numpy())


In [ ]:
output = {'ID': [int(x) for x in no_label[b'ids']], 'Labels': [int(x) for x in predictions]}

In [ ]:
with open(f'/scratch/nd2745/DL_Project_1/results/output_{model_name}_transformed.csv', mode='w', newline='') as file:
    writer = csv.writer(file)
    
    # Write the header (keys of the dictionary)
    writer.writerow(output.keys())
    
    # Write the rows (values of the dictionary, zipped together)
    writer.writerows(zip(*output.values()))

# Optional: Print confirmation
print("CSV file has been created successfully!")